# Code for Selecting Similar Corpus
## use doc2vec

First read all the data turn them to dic, vectorize all data and build the model. Second, infer a special corpus you want to caculate the similarity. And the result is print out 

In [3]:
import os
import gensim
import os
import collections
import smart_open
import random

#extract training files
path = "/home/work/SSData/en_zh-cn/"

def get_name(path):
    allfile = []
    for p in path:
        allfile.extend([os.path.join(p,fn) for fn in next(os.walk(p))[2]])
    return allfile
def get_dir(path):
    dirs = [os.path.join(path,fn) for fn in next(os.walk(path))[1]]
    return dirs

train_file = [f for f in get_name(get_dir(path)) if f.split(".")[1] == "en"]


StopIteration: 

In [36]:
def read_corpus(train_file, tokens_only=False):
    for ix,t in enumerate(train_file):
        onelinefile = ""
        with smart_open.smart_open(t, encoding="utf-8") as f:
            for line in f:
                if len(line.strip()) !=0:
                    onelinefile += line
        if tokens_only:
            yield gensim.utils.simple_preprocess(onelinefile)
        else:
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(onelinefile), [ix])
                
            

train_corpus = list(read_corpus(test_file))

len(train_corpus)

33

In [38]:
#intialize model and train the corpus
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=20)
model.build_vocab(train_corpus)
%time model.train(train_corpus)

CPU times: user 44.5 s, sys: 4min 8s, total: 4min 53s
Wall time: 25 s


2816180

In [41]:
#find out the similarity about a specific file
inferred_vector = model.infer_vector(train_corpus[0].words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
print('Document ({}): «{}»\n'.format(1, ' '.join(train_corpus[0].words[:200])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', 1), ('LEAST', 2)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words[:200])))


Document (1): «party needs more copies if there is necessary need of construction it shall get the approval of the owner party expenses shall be borne to party notification of defects commercial operation period warranty period engineering units means the component of the individual works can construct independently headings and marginal notes of each clause only for the convenience of reading shall not be as part of clause this contract document is written and explained in chinese the laws and regulations of the people republic of china and the laws and regulations of myanmar shall apply this contract document the contract price is rmb upper case contract no the above drawings provided to party or drawings provided by its own party shall keep at the construction site notifications of award means the written notice party formally grants the construction contract to party management of equipments and materials in worksite supplementary drawings and instructions when replacing those pers

## Try kmeans with test data file

In [5]:
from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

In [19]:
path_test="/home/work/langData/en_zh-cn/electronic/"
test_file = [f for f in get_name(get_dir(path_test)) if f.split(".")[1] == "en"]
#for fil in test_file:
   # print fil.decode('utf-8')
def read(test_file):
    corpus = []
    for t in test_file:
        line = ""
        with smart_open.smart_open(t, encoding="utf-8") as f:
            for l in f:
                if len(l.strip())>0:
                    line += l.strip()
        corpus.append(line)
    return corpus

test_corpus = read(test_file)
len(test_corpus)

33

In [7]:
vectorizer=TfidfVectorizer(min_df=1, max_df=0.9, stop_words='english', decode_error='ignore')
vectorized=vectorizer.fit_transform(test_corpus)

km=KMeans(n_clusters=4, init='k-means++',n_init=10, verbose=1)
km.fit(vectorized)
pred = km.predict(vectorized)



Initialization complete
Iteration  0, inertia 47.100
Iteration  1, inertia 25.100
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.432620e-10
Initialization complete
Iteration  0, inertia 45.461
Iteration  1, inertia 25.213
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.432620e-10
Initialization complete
Iteration  0, inertia 43.878
Iteration  1, inertia 24.512
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.432620e-10
Initialization complete
Iteration  0, inertia 43.465
Iteration  1, inertia 24.599
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.432620e-10
Initialization complete
Iteration  0, inertia 40.895
Iteration  1, inertia 25.084
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.432620e-10
Initialization complete
Iteration  0, inertia 42.293
Iteration  1, inertia 25.173
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.432620e-10
Initialization c

In [8]:
vectorized.shape

(33, 253205)

In [9]:
pred

array([0, 1, 3, 3, 3, 3, 1, 3, 3, 0, 3, 2, 1, 3, 0, 3, 1, 3, 1, 0, 3, 1, 2,
       3, 3, 1, 1, 3, 1, 1, 3, 1, 1], dtype=int32)

In [35]:
ca = []
cb = []
cc = []
cd = []
for p in range(len(pred)-1):
    if pred[p]==0:
        ca.append(p)
    elif pred[p]==1:
        cb.append(p)
    elif pred[p]==2:
        cc.append(p)
    else:
        cd.append(p)
def content(fl):
    with open(fl) as inp:
        print (inp.read()[:1000],"\n")
    
for c in ca:
    print (content(test_file[c].decode('utf-8')))
    


Party B needs more copies if there is a necessary need of construction, it shall get the approval of the owner/ Party A, expenses shall be borne to Party B.
Notification of defects (commercial operation period/ warranty period):
“Engineering Units” means the component of the individual works, can construct independently.
Headings and marginal notes of each clause only for the convenience of reading, shall not be as a part of clause.
This contract document is written and explained in Chinese, the laws and regulations of the People's Republic of China and the laws and regulations of Myanmar shall apply this contract document.
The contract price is RMB(upper case):
Contract No.:
The above drawings provided to Party B or drawings provided by its own, Party B shall keep at the construction site.
“Notifications of Award” means the written notice Party A formally grants the construction contract to Party B.
/ /2016
Management of Equipments and Materials in a Worksite
Supplementary Dra 

None
